In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Load the DataFrame
df = pd.read_csv("soilmoisture_dataset.csv", index_col=0)

# Remove empty rows
df = df.dropna()

# Reset the index without adding the old index as a column
df.reset_index(drop=True, inplace=True)

# Drop unnecessary columns
df = df.drop(['soil_temperature', 'datetime'], axis=1)

# Split 90% of the data into `train_val` and 10% into `test`
train_val, test = train_test_split(df, test_size=0.2, random_state=42)

X_cross_val = train_val.drop('soil_moisture', axis=1)
y_cross_val = train_val['soil_moisture']

X_test2 = test.drop('soil_moisture', axis=1)
y_test2 = test['soil_moisture']

In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from math import sqrt

# Define the adjusted R-squared function
def adjR2(r2, n_features, n_samples):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Create a RandomForestRegressor model for feature selection
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=15)

# Create an SVR model for training and evaluation
svr_model = SVR(kernel='rbf', C=10000, gamma='scale')

# Train the Random Forest Regressor on the entire dataset to determine feature importance
rf_model.fit(X_cross_val, y_cross_val)

# Get feature importances
feature_importances = rf_model.feature_importances_

# Create a DataFrame of features and their importances
feature_importance_df = pd.DataFrame({
    'Feature': X_cross_val.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select the top 15 features
top_features = feature_importance_df['Feature'].head(15).tolist()

print("Top 15 Features Selected:", top_features)

# Initialize a list to store results
results = []

# Evaluate performance for top 1 to 15 features
for n in range(1, len(top_features) + 1):
    # Select the top `n` features
    selected_features = top_features[:n]

    print(f"Using top {n} features:", selected_features)

    # Perform 10-fold cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    rmse_list = []
    adjR_list = []

    for train_index, val_index in kf.split(X_cross_val):
        # Split the data into training and validation sets for this fold
        X_train, X_val = X_cross_val.iloc[train_index][selected_features], X_cross_val.iloc[val_index][selected_features]
        y_train, y_val = y_cross_val.iloc[train_index], y_cross_val.iloc[val_index]

        # Scale the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform on training data
        X_val_scaled = scaler.transform(X_val)  # Transform validation data

        # Train the SVR model on the scaled data
        svr_model.fit(X_train_scaled, y_train)
        y_pred = svr_model.predict(X_val_scaled)

        # Calculate metrics
        mse = mean_squared_error(y_val, y_pred)
        rmse = sqrt(mse)
        r2 = r2_score(y_val, y_pred)
        adj_r2 = adjR2(r2, n, len(y_val))

        # Append metrics for this fold
        rmse_list.append(rmse)
        adjR_list.append(adj_r2)

    # Average metrics across folds
    avg_rmse = sum(rmse_list) / len(rmse_list)
    avg_adj_r2 = sum(adjR_list) / len(adjR_list)

    # Append the results
    results.append({
        'Top Features': n,
        'Selected Columns': ', '.join(selected_features),
        'Average RMSE': round(avg_rmse, 4),
        'Average Adjusted R²': round(avg_adj_r2, 4)
    })

    print(f"Top {n} Features -> Avg RMSE: {avg_rmse:.4f}, Avg Adj R²: {avg_adj_r2:.4f}")

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to Excel
results_df.to_excel("RF_TopFeatures_SVR_results.xlsx", index=False)

print("Results saved to 'RF_TopFeatures_SVR_results.xlsx'")


Top 15 Features Selected: ['950', '946', '786', '942', '454', '458', '494', '858', '826', '790', '770', '782', '794', '498', '914']
Using top 1 features: ['950']
Top 1 Features -> Avg RMSE: 1.9052, Avg Adj R²: 0.7024
Using top 2 features: ['950', '946']
Top 2 Features -> Avg RMSE: 1.8831, Avg Adj R²: 0.7033
Using top 3 features: ['950', '946', '786']
Top 3 Features -> Avg RMSE: 1.6642, Avg Adj R²: 0.7607
Using top 4 features: ['950', '946', '786', '942']
Top 4 Features -> Avg RMSE: 1.6835, Avg Adj R²: 0.7511
Using top 5 features: ['950', '946', '786', '942', '454']
Top 5 Features -> Avg RMSE: 1.2814, Avg Adj R²: 0.8449
Using top 6 features: ['950', '946', '786', '942', '454', '458']
Top 6 Features -> Avg RMSE: 1.1305, Avg Adj R²: 0.8789
Using top 7 features: ['950', '946', '786', '942', '454', '458', '494']
Top 7 Features -> Avg RMSE: 0.9178, Avg Adj R²: 0.9139
Using top 8 features: ['950', '946', '786', '942', '454', '458', '494', '858']
Top 8 Features -> Avg RMSE: 0.8713, Avg Adj R²: